# Module 4 R Notebook
# Predicting Diseases from Genes

## Important note - read first

In this course, you are not required to become proficient in writing R scripts for Big Biomedical Data Analytics. The assignments will be based on each module's template, and replacing some parts of them with new data and parameters so that you will get new results and interpret them. 

Understanding all the coding details in R will depend on your bakcground knowledge in programming concepts. 

Therefore if you do not understand all the coding details in R, try to run the code and understand mostly: 

1) what information do I have to provide to this script to make it work ?

2) how to execute this script (it will always be: run the cell containing the script Cell --> Run cells, or click on SHIFT + ENTER at the same time, or simply click on the right arrow icon) ?

3) what results do I get ?

4) how to interpret the results ?

Little by little, you will become familiar with R programming language and, example after example, you will understand the scripts and be ready for more advanced programming work. 


## Assignment goals

In this notebook, you are going to practice classification / prediction with R scripts, and particularly prediction based on gene expressions. This implies some data preprocessing then building a classification / prediction model to predict a disease (for example, this same process could be applied to different types of prediction).

Here we are continuing to work on a breast cancer dataset downloaded from Firehose (https://gdac.broadinstitute.org/). These are next generation sequencing data that are provided already normalized. 

We are going to compare two classifiers - KNN (K-nearest neighbor) and RandomForest (a classifier in the rule / tree family).

Each time we are going to build the models on the top 100 genes selected through bss/wss in Module 3, once using the training set as a test set (most favorable scenario), and next using the training set as 70% of the whole dataset, and the test set as 30% (least favorable scenario). We expect the classification accuracy to drop when moving from testing on the training set and training on 70% of the data. But how much is the accuracy goin to drop ? And which of the two models is going to perform the best ? These are the questions we will try and answer.  

So let us get started !

## Preparing the environment

The processes employed to build the predictive model use quite a bit of RAM memory, therefore we first allocate this memory by increasaing the program memory size to 3.5GB (= 3,500 MB). We also load some libraries used for analysis - 'class' and 'randomForest'.

In [3]:
install.packages("randomForest")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [5]:
install.packages("class")

also installing the dependency ‘MASS’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [7]:
# cell #1
#memory.limit(size=3500)
library(randomForest)
library(class)

## Loading the data

We again load the data as we did in module 3. We create a dataframe (a spreadsheet) 'mrnaNorm' with the gene expression values and the first column being the gene names. The second dataframe 'mrnaIDs' contains the IDs of the patients. 

In [8]:
# cell #2

mrnaNorm <- read.table("BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt", 
            header = F, fill = T, skip = 2)
mrnaIDs <- read.table("BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt", 
            header = F, fill = T, nrows = 1)
mrnaIDs <- mrnaIDs[, -1][, -1]

In [10]:
mrnaNorm[1:5,1:5]
mrnaIDs

V1,V2,V3,V4,V5
?|100130426,0.0000,0.0000,0.9066,0.0000
?|100133144,16.3644,9.2659,11.6228,12.0894
?|100134869,12.9316,17.3790,9.2294,11.0799
?|10357,52.1503,69.7553,154.2974,143.8643
?|10431,408.0760,563.8934,1360.8341,865.5358


V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V1205,V1206,V1207,V1208,V1209,V1210,V1211,V1212,V1213,V1214
TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SD-01A-11R-A115-07,⋯,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-V7-A7HQ-01A-11R-A33J-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07


## Data preprocessing

'mrnaClass' and 'mrnaClassNum' are created and contain the diagnostic class - 0 for normal and 1 for tumor.

In [13]:
# cell #3
samp <- lapply(as.list(t(mrnaIDs)), function(t) substr(unlist(strsplit(t, "-"))[4], 1, 2))
sampleType <- as.data.frame(samp)
sampClass <- lapply(samp, function(t) (if (t < 10) return("1") else return("0")))
mrnaClass <- as.data.frame(sampClass)
dim(mrnaNorm)
# 20531 1213 columns are patients (except the 1st for gene name) rows are expression levels for each gene
dim(mrnaIDs)
# 1 1213   the first column is the gene name, the others are one patient per row
dim(mrnaClass)
# 1 1212 one patients per row   1 = tumor, 0 = normal
table(unlist(sampClass))
#   0    1 
# 112 1100                     112 normals and 1100 tumor
sampClassNum <- lapply(samp, function(t) (if (t < 10) return(1) else return(0)))
mrnaClassNum <- as.data.frame(sampClassNum) 

[1] 20531  1213

[1]    1 1212

[1]    1 1212


   0    1 
 112 1100 

In [15]:
head(mrnaClassNum)

X1,X1.1,X1.2,X1.3,X1.4,X1.5,X1.6,X1.7,X1.8,X1.9,⋯,X1.1090,X1.1091,X1.1092,X1.1093,X1.1094,X1.1095,X1.1096,X1.1097,X1.1098,X1.1099
1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1


We also create a dataframe with only the gene names, called 'geneNames', which are located in the first column of 'mrnaNorm', which we extract.

In [16]:
# cell #4
geneNames <- mrnaNorm[1] # extract the gene names from mrnaNorm as its first column
dim(geneNames)
# 20531 genes
head(geneNames)

[1] 20531     1

V1
?|100130426
?|100133144
?|100134869
?|10357
?|10431
?|136542


First we transpose the 'mrnaNorm' dataframe because we want to select genes, therefore they have to be represented in columns instead of rows. We also remove the first column of 'mrnaNorm' since it contains the gene names.
Because we are working with large datasets, we free space from memory by removing the objects we will not be using anymore. 'gc' garbage collects the free space, which will leave more space for building the models. The column 'used' and right after it '(Mb)' indicates the memory in use.

In [17]:
# cell #5
mrnaData = t(mrnaNorm[, -1]) # remove first column of mrnaData and transpose it to have genes as columns
rm(samp)
rm(sampClass)
rm(mrnaNorm)
gc()
mrnaData[1:5,1:5]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,613945,32.8,6719724,358.9,8046738,429.8
Vcells,30244750,230.8,98117289,748.6,80064801,610.9


V2,0.0000,16.3644,12.9316,52.1503,408.0760
V3,0.0000,9.2659,17.3790,69.7553,563.8934
V4,0.9066,11.6228,9.2294,154.2974,1360.8341
V5,0.0000,12.0894,11.0799,143.8643,865.5358
V6,0.0000,6.8468,14.4298,84.2128,766.3830


## Classification / prediction on all features

It is possible to run the classification model on all 20,531 genes, however the processing lasts several minutes. For this reason, we are going to first select features before the classification to make it very efficient. 

The code below runs KNN on all features, however I am going to skip it because it takes too long to run. If you are interested, you can test it. I have placed the '#' synbol in front of all the lines, you can remove it to run this cell (leave the one in front of the first line though).

In [19]:
# cell #6
trainSet <- mrnaData
testSet <- mrnaData
trainClasses <- unlist(mrnaClassNum[1,], use.names=FALSE)
testClasses <- unlist(mrnaClassNum[1,], use.names=FALSE)
knn.predic <- knn(trainSet, testSet, trainClasses, testClasses, k=1)
cbr.predic = as.vector(knn.predic)
table(cbr.predic, testClasses)
tab <- table(cbr.predic, t(testClasses))
error <- sum(tab) - sum(diag(tab))
accuracy <- round(100- (error * 100 / length(testClasses)))
print(paste("accuracy= ", as.character(accuracy), "%"), quote=FALSE)

          testClasses
cbr.predic    0    1
         0  112    0
         1    0 1100

[1] accuracy=  100 %


## Feature selection with bss/wss

Again, we select top genes with bss/wss. First we load the bssWssFast function.

In [20]:
# cell #7
bssWssFast <- function (X, givenClassArr, numClass=2)
# between squares / within square feature selection
{
	classVec <- matrix(0, numClass, length(givenClassArr))
	for (k in 1:numClass) {
		temp <- rep(0, length(givenClassArr))
		temp[givenClassArr == (k - 1)] <- 1
		classVec[k, ] <- temp
	}
	classMeanArr <- rep(0, numClass)
	ratio <- rep(0, ncol(X))
	for (j in 1:ncol(X)) {
		overallMean <- sum(X[, j]) / length(X[, j])
		for (k in 1:numClass) {
			classMeanArr[k] <- 
				sum(classVec[k, ] * X[, j]) / sum(classVec[k, ])
		}
	  classMeanVec <- classMeanArr[givenClassArr + 1]
	  bss <- sum((classMeanVec - overallMean)^2)
	  wss <- sum((X[, j] - classMeanVec)^2)
	  ratio[j] <- bss/wss
	}
      sort(ratio, decreasing = TRUE, index = TRUE)
}

Then we again run 'bssWssFast' on our large dataset to rank the features within and across classes. We work from mrnaData, which is the transposed datafrom from 'mrnaNorm' dataframe because we want to select genes, therefore they have to be represented in columns instead of rows.

In [32]:
# cell #8
# select features
dim(mrnaData)
# 1212 20531  matrix
dim(mrnaClass)
# 1 1212
dim(mrnaClassNum)
# 1 1212
dim(geneNames)
# 20531 genes
bss <- bssWssFast(mrnaData, t(mrnaClassNum), 2)


[1]  1212 20531

[1]    1 1212

[1]    1 1212

[1] 20531     1

## Classification / prediction on selected features on training set

First we are going to build a classification model on the complete dataset as a training set, then test its performance on the same complete dataset. We call that testing on the training set. This is the easiest task for a model. It is as though we gave students the test in advance and tested them during an exam on the same tests they had studied. 

We extract the top 100 gene expressions and place them in 'mrnaDataReduced'.

We create a 'trainSet' and 'testSet' which are identical because we are going to test the model on the training set.

We extract the 'trainClasses' and 'testClasses' from mrnaClassNum. Since classification / prediction is a supervised model, we give the models the classes we know for training, then use these classes for testing whether a model predicts the right class.

In [33]:
# cell #9
mrnaDataReduced <- mrnaData[,bss$ix[1:100]]
dim(mrnaDataReduced)
# 1212  100
trainSet <- mrnaDataReduced
testSet <- mrnaDataReduced
trainClasses <- unlist(mrnaClassNum[1,], use.names=FALSE)
# or as.numeric(mrnaClassNum[1,])
testClasses <- unlist(mrnaClassNum[1,], use.names=FALSE)

[1] 1212  100

In [34]:
trainClasses[5:1]

[1] 1 1 1 1 1

### KNN on selected features on training set

First, we are going to build a KNN model on this reduced dataset of 100 genes and 1212 patients / normals with 112 normals and 1100 tumor patients.

We pass to the 'knn' function, from the 'class' package, a training set, a test set, the list of classes for the training set, and the lsit of classes for the test set.

We then build the confusion matrix as: 

            testClasses
knn.predic    0    1
         0  112    0
         1    0 1100

which shows that from test classes of value 0, 112 are predicted as value 0, and from the test classes of value 1, 1100 are predicted as 1. 
This yields an accuracy of 100%, being calculated as TP + TN / TP + FP + TN + FN = (1100 + 112) / (1100 + 0 + 112 + 0).

It cannot get better than that - very fast, perfect result.

In [35]:
# cell #10
knn.predic <- knn(trainSet, testSet, trainClasses, testClasses,k=1) # knn form 'class' package
knn.predic = as.vector(knn.predic)  # change knn.predic to become a vector
table(knn.predic, testClasses)      # build the confusion matrix
tab <- table(knn.predic, t(testClasses))
error <- sum(tab) - sum(diag(tab))  # calculate acuracy
accuracy <- round(100- (error * 100 / length(testClasses)))
print(paste("accuracy= ", as.character(accuracy), "%"), quote=FALSE)   # display acuracy after formating it as a character string

          testClasses
knn.predic    0    1
         0  112    0
         1    0 1100

[1] accuracy=  100 %


### RandomForest on selected features on training set

Now let us compare the same easy scenario with RandomForest from 'randomForest' package.

The 'rf' function, which builds the RandomForest, takes slightly different arguments than 'knn'.

First, instead of having separate gene expressions set and diagnostic class set, it takes only one dataframe. Therefore we concatenate with 'cbind' the gene expressions and the diagnostic class 0/1.

Next, the target class needs to have a name, which we choose as 'class'. 

In addition, for classification, the target variable, here 'class', needs to be categorical, which is obtained by applying 'as.factor'.

Finally, there are two steps. First building the model with 'randomForest', then testing it on the test set with 'predict'.

We run, and bingo, we get another 100% acuracy. 

So KNN and RandomForest are tied on the training set. 

We are going to break the tie by using a different training set and test set. In other words, we are going to raise the bar in difficulty. It is like in a sport competition.

In [36]:
# cell #11
trainSetClass <- as.data.frame(cbind(trainSet, t(mrnaClassNum[1,])))  # concatenate gene expressions and class data
testSetClass <- as.data.frame(cbind(testSet, t(mrnaClassNum[1,])))    # concatenate gene expressions and class data
colnames(trainSetClass)[101] <- "class"     # give a name to the class column
#trainSetClass$class <- as.numeric(trainSetClass$class) # for regression
trainSetClass$class <- as.factor(trainSetClass$class)  # for classification
class(trainSetClass$class)      # should be factor or categorical for classification
rf <- randomForest(class ~., trainSetClass,
                              ntree=100,
                              importance=T)      # build randomForest classifier
colnames(testSetClass)[101] <- "class"     # give a name to the class column
testSetClass$class <- as.factor(testSetClass$class)  # for classification
rf.predic <- predict(rf ,testSetClass)  # test the randomForest built model on the test set
rf.predic = as.vector(rf.predic)        # change rf.predic to become a vector
table(rf.predic, testClasses)           # build the confusion matrix
tab <- table(rf.predic, t(testClasses))
error <- sum(tab) - sum(diag(tab))      # calculate acuracy
accuracy <- round(100- (error * 100 / length(testClasses)))
print(paste("accuracy= ", as.character(accuracy), "%"), quote=FALSE)

[1] "factor"

         testClasses
rf.predic    0    1
        0  112    0
        1    0 1100

[1] accuracy=  100 %


## Classification / prediction on selected features on independent test set

In this experiment, we are first going to separate the dataset into two sets:

1) a training set composed of 70% of the data samples
2) a test set composed of 30% of the data samples.

After that, all the steps will be the same and we are going to compare classification accuracy between KNN and RandomForest.

To split the dataset, there is a 'sample' function in R which here will take 70% of the rows. The rest will be the test set.

Therefore our training set will have 848 rows, and our test set will have 364 rows.

In [49]:
# cell #12
nbRows <- nrow(mrnaDataReduced)
set.seed(22)       # seet random seed so that we always get same samples drawn - since they are random
trainRows <- sample(1:nbRows, .70*nbRows)
trainSet <- mrnaDataReduced[trainRows, ]
testSet <- mrnaDataReduced[-trainRows, ]
dim(trainSet)
dim(testSet)

[1] 848 100

[1] 364 100

#### KNN on selected features on independent test set

We run again KNN on the different training and test sets and obtain a classification accuracy of 99%. We have 5 errors on 364 data samples, which is a very high result. The exact figure is:

360 / 364 = 98.90%

In [50]:
# cell #13
trainClasses <- unlist(mrnaClassNum[1,trainRows], use.names=FALSE)
testClasses <- unlist(mrnaClassNum[1,-trainRows], use.names=FALSE)
knn.predic <- knn(trainSet, testSet, trainClasses, testClasses,k=1)
knn.predic = as.vector(knn.predic)
table(knn.predic, testClasses)
tab <- table(knn.predic, t(testClasses))
error <- sum(tab) - sum(diag(tab))
accuracy <- round(100- (error * 100 / length(testClasses)))
print(paste("accuracy= ", as.character(accuracy), "%"), quote=FALSE)

          testClasses
knn.predic   0   1
         0  36   3
         1   1 324

[1] accuracy=  99 %


### RandomForest on selected features on independent test set

We run again RandomForest on the different training and test sets and obtain a classification accuracy of 99%. We have 3 errors on 364 data samples, which is a very high result. The exact figure is:

361 / 364 = 99.18%

Therefore once more KNN and RandomForest are tied. However we notice that the errors are not exactly the same. RandomForest has no misclassified '1', while KNN has three '1' classifed as '0' (1 = tumor, 0 = normal). This means than in three cases, patients with a tumor are classified as being normal, which is what is called a false negative. In a medical domain, this type of error is more severe than the false positive because some patients will think that they are fine and not seek care while they should seek care.

In [51]:
# cell #14
trainSetClass <- as.data.frame(cbind(trainSet, t(mrnaClassNum[1,trainRows])))
testSetClass <- as.data.frame(cbind(testSet, t(mrnaClassNum[1,-trainRows])))
colnames(trainSetClass)[101] <- "class"
trainSetClass$class <- as.factor(trainSetClass$class)  # for classification
class(trainSetClass$class)
# should be factor for classification
rf <- randomForest(class ~., trainSetClass,
                              ntree=100,
                              importance=T)
colnames(testSetClass)[101] <- "class"
testSetClass$class <- as.factor(testSetClass$class)  # for classification
rf.predic <- predict(rf ,testSetClass)
rf.predic = as.vector(rf.predic)
table(rf.predic, testClasses)
tab <- table(rf.predic, t(testClasses))
error <- sum(tab) - sum(diag(tab))
accuracy <- round(100- (error * 100 / length(testClasses)))
print(paste("accuracy= ", as.character(accuracy), "%"), quote=FALSE)

[1] "factor"

         testClasses
rf.predic   0   1
        0  34   0
        1   3 327

[1] accuracy=  99 %


This is just a small example of what is possible to do. Tehre are hundreds of classification / prediction models, and same for feature selection methods. By combining them, we obtained thousands of scenarios for building a classification / prediction model. The two models selected above are particularly efficient and effective as we could see.               